In [1]:
%matplotlib notebook
import time
import sklearn
import pandas as pd
import numpy as np
import category_encoders
from numpy.linalg.linalg import LinAlgError
from collections import Counter
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression as lr
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as confu
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.cluster import KMeans
from itertools import combinations as Cb
from sklearn.tree import ExtraTreeClassifier as ex_tree


/usr/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Normalize data
def normalization(training_numpy, testing_numpy):
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy)
    testing_x_scaled = min_max.transform(testing_numpy)
    #training_norm = pd.DataFrame(x_scaled, columns = columns)
    #testing_norm = pd.DataFrame(testting_x_scaled, columns = columns)
    return (x_scaled, testing_x_scaled)

def pca_function(rate, data):
    pca = PCA()
    pca.fit(data)
    for thres_n in xrange(1,len(data)):
        if sum(pca.explained_variance_ratio_[:thres_n])>rate:
            pca_n = thres_n
            break
    
    pca = PCA(n_components=pca_n)
    pca_data = pca.fit_transform(data)
    return (pca_data, pca)

def lda_whole_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


def lda_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


'''
def pca_function(n, data):
    pca = PCA(n_components=n)
    pca_data = pca.fit_transform(data)
    return (pca_data.flatten(), pca)

def lda_function(X, y):
    lda = LDA()
    lda_data = lda.fit_transform(X, y)
    return (lda_data.flatten(), lda)
'''
def replace_value(training_normal_numpy, key, loc):  
    training_normal_numpy[loc][key] = 1
    
def del_zero_column(df_train, df_test):
    new_df_test = pd.DataFrame(np.zeros([df_test.shape[0], df_train.shape[1]]),columns=df_train.columns)
    new_df_test[list(set(df_train.columns) & set(df_test.columns))] = df_test[list(set(df_train.columns) & set(df_test.columns))]
    #df_test = df_test[df_train.columns]
    return (df_train, new_df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]
#def del_zero_column(df_train, df_test):
#    df_train = df_train.loc[:, (df_train != 0).any(axis=0)]
#    df_test = df_test[df_train.columns]
#    return (df_train, df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]

def all_lda(train_combine, label_num, n_cluster = 2):
    train_all_dummy = pd.get_dummies(train_combine)
    #train_all_dummy = category_encoders.OneHotEncoder(cols=train_combine.columns.tolist()).fit_transform(train_combine)
    (train_lda, lda) = lda_whole_function(train_all_dummy, np.array(label_num), n_cluster=n_cluster)
    clf=BernoulliNB()
    start = time.time()
    
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score , time.time()-start)

def divide_lda(train_combine, label_num, n_cluster = 2):
    train_column= train_combine.columns
    for col in train_combine.columns:
        train_dummy.append(pd.get_dummies(train_combine[col]))
        #train_dummy.append(category_encoders.OneHotEncoder(cols=[col]).fit_transform(train_combine[[col]]))
    train_lda = pd.DataFrame(index=range(len(label_num)))
    for i in range(len(train_combine.columns)):
        #print train_dummy[i].shape
        (now_train_lda, lda) = lda_whole_function(train_dummy[i], np.array(label_num), n_cluster=3)
        now_lda_col_name = []
        for j in range(len(now_train_lda[0])):
            train_lda[train_column[i]+'_'+str(j+1)] = np.array(now_train_lda)[:,j]
    start = time.time()
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score, time.time()-start)

 

# DNA

In [3]:
def combine_selection(train_dummy, label):
    columns = train_dummy.columns
    feature_dic = {}
    new_column = []
    initial = 1
    whole_train_dummy = pd.DataFrame()
    for column, idx in zip(columns, range(len(columns))):     
        now_column = []
        each_train_dummy = pd.get_dummies(train_dummy[column])
        
        clf = ex_tree()
        clf.fit(each_train_dummy, label)
        
        for key, value in zip(each_train_dummy.columns, clf.feature_importances_):
            now_column.append(('combine%d_'+key) % idx)
            new_column.append(('combine%d_'+key) % idx)
            feature_dic[('combine%d_'+key) % idx] = value
        each_train_dummy.columns = now_column
        if initial == 1:
            whole_train_dummy = whole_train_dummy.append(each_train_dummy)
            initial = 0
        else:
            whole_train_dummy = whole_train_dummy.join(each_train_dummy)
        
    #print len(new_column)
    sorted_dic = sorted(feature_dic.iteritems(), key=lambda (k,v): (v,k),reverse=True)
    
        
    
    return (sorted_dic, whole_train_dummy) 

In [ ]:
# Do not run yet
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])
print train.shape


total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

highest_score = 0
highest_idx = 0
exp_num = 0
for combine_num in range(2,3):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        train_label_list.append(val_label[train_val_index])
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
        print "top: ", threshold
        val_time = []
        now_score = []
        for cv in xrange(10):   

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
            train_dummy_top = train_dummy_list[cv][sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
            test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv])        
                clf.fit(train_lda, train_label_list[cv])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), test_label_list[cv]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
                
                
        exe_time = np.sum(val_time)
        exp_num = exp_num + 1
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        
        if np.average(now_score) > highest_score:
            highest_score = np.average(now_score)
            highest_idx = exp_num
        
        result.to_csv('dna_combine_lda_normal.csv')
print "Highest:", result.loc[highest_idx]

(3190, 60)


# Mushroom

In [ ]:
mush_columns = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
               'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',\
               'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type',\
               'spore-print-color', 'population', 'habitat']
train = pd.read_csv('agaricus-lepiota.data',header=None)
train.columns = mush_columns
key_str = {'e':0, 'p':1}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','top','score', 'time'])


total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)


highest_score = 0
highest_idx = 0
exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        train_label_list.append(val_label[train_val_index])
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
        print "top: ", threshold
        val_time = []
        now_score = []
        for cv in xrange(10):   

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
            train_dummy_top = train_dummy_list[cv][sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
            test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv])        
                clf.fit(train_lda, train_label_list[cv])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), test_label_list[cv]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
                
                
        exe_time = np.sum(val_time)
        exp_num = exp_num + 1
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        
        if np.average(now_score) > highest_score:
            highest_score = np.average(now_score)
            highest_idx = exp_num
        
        result.to_csv('mush_combine_lda_normal.csv')
print "Highest:", result.loc[highest_idx]

# Car

In [52]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

highest_score = 0
highest_idx = 0
exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        train_label_list.append(val_label[train_val_index])
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
        print "top: ", threshold
        val_time = []
        now_score = []
        for cv in xrange(10):   

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
            train_dummy_top = train_dummy_list[cv][sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
            test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv])        
                clf.fit(train_lda, train_label_list[cv])
                test_lda = lda.transform(test_dummy_top)
                start = time.time()
                now_score.append(accuracy_score(clf.predict(test_lda), test_label_list[cv]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
                
                
        exe_time = np.sum(val_time)
        exp_num = exp_num + 1
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        
        if np.average(now_score) > highest_score:
            highest_score = np.average(now_score)
            highest_idx = exp_num
        
        result.to_csv('car_combine_lda_normal.csv')
print "Highest:", result.loc[highest_idx]

combine_subset: 15
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2
combine_num 2
score: 0.719351304707
time: 0.0298433303833
top:  4
combine_num 2
score: 0.615562093642
time: 0.0241522789001
top:  6
combine_num 2
score: 0.605584814839
time: 0.0232334136963
top:  8
combine_num 2
score: 0.593395560629
time: 0.0221705436707
top:  10
combine_num 2
score: 0.603813550994
time: 0.0216951370239
top:  12
combine_num 2
score: 0.596836806808
time: 0.0213060379028
top:  14
combine_num 2
score: 0.594534867999
time: 0.0207333564758
top:  16
combine_num 2
score: 0.595690975991
time: 0.0205519199371
top:  18
combine_num 2
score: 0.598347487846
time: 0.0190458297729
top:  20
combine_num 2
score: 0.605351393864
time: 0.0160489082336
top:  22
combine_num 2
score: 0.670471424628
time: 0.0160624980927
top:  24
combine_num 2
score: 0.685471087409
time: 0.0159206390381
top:  26
combine_num 2
score: 0.671691350049
time: 0.0151250362396
top:  28
combine_num 2


# Bot

In [44]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_5.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        train_label_list.append(val_label[train_val_index])
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
        print "top: ", threshold
        val_time = []
        now_score = []
        for cv,(train_val_index,test_val_index) in zip(xrange(10),skf):   

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
            train_dummy_top = train_dummy_list[cv][sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
            test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv]) 
                train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                test_lda = lda.transform(test_dummy_top)
                test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                (train_lda, test_lda) = normalization(train_lda, test_lda)
                clf.fit(train_lda, train_label_list[cv])
                
                start = time.time()
                now_score.append(f1_score(clf.predict(test_lda), test_label_list[cv]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exp_num = exp_num + 1
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        result.to_csv('bot5_combine_lda_normal.csv')

combine_subset: 6
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2
combine_num 2
score: 0.424562628188
time: 0.121381044388
top:  4
combine_num 2
score: 0.657585950135
time: 0.0998449325562
top:  6
combine_num 2
score: 0.691196473541
time: 0.0931324958801
top:  8
combine_num 2
score: 0.772398478871
time: 0.0764217376709
top:  10
combine_num 2
score: 0.763696811958
time: 0.0689964294434
top:  12
combine_num 2
score: 0.80006120156
time: 0.0670890808105
top:  14
combine_num 2
score: 0.807366560509
time: 0.057196855545
top:  16
combine_num 2
score: 0.802856752206
time: 0.0558042526245
top:  18
combine_num 2
score: 0.805660990913
time: 0.0552294254303
top:  20
combine_num 2
score: 0.805660990913
time: 0.0549538135529
top:  22
combine_num 2
score: 0.807126375369
time: 0.0550079345703
top:  24
combine_num 2
score: 0.811999927384
time: 0.0544443130493
top:  26
combine_num 2
score: 0.818433284459
time: 0.0531058311462
top:  28
combine_num 2
scor

In [45]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_6.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        train_label_list.append(val_label[train_val_index])
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
        print "top: ", threshold
        val_time = []
        now_score = []
        for cv,(train_val_index,test_val_index) in zip(xrange(10),skf):   

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
            train_dummy_top = train_dummy_list[cv][sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
            test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv]) 
                train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                test_lda = lda.transform(test_dummy_top)
                test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                (train_lda, test_lda) = normalization(train_lda, test_lda)
                clf.fit(train_lda, train_label_list[cv])
                
                start = time.time()
                now_score.append(f1_score(clf.predict(test_lda), test_label_list[cv]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exp_num = exp_num + 1
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        result.to_csv('bot6_combine_lda_normal.csv')

combine_subset: 6
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2
combine_num 2
score: 0.974403786759
time: 0.112164735794
top:  4
combine_num 2
score: 0.974290850288
time: 0.10874080658
top:  6
combine_num 2
score: 0.974290850288
time: 0.107858657837
top:  8
combine_num 2
score: 0.981092545309
time: 0.0865757465363
top:  10
combine_num 2
score: 0.986768133949
time: 0.066107749939
top:  12
combine_num 2
score: 0.99353742595
time: 0.0435831546783
top:  14
combine_num 2
score: 0.996313531596
time: 0.0327143669128
top:  16
combine_num 2
score: 0.996313531596
time: 0.0279021263123
top:  18
combine_num 2
score: 0.996313530579
time: 0.0281000137329
top:  20
combine_num 2
score: 0.996313530579
time: 0.0285348892212
top:  22
combine_num 2
score: 0.996205658249
time: 0.0278301239014
top:  24
combine_num 2
score: 0.996205658249
time: 0.0286920070648
top:  26
combine_num 2
score: 0.996205658249
time: 0.0283064842224
top:  28
combine_num 2
score: 

In [47]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_7.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        train_label_list.append(val_label[train_val_index])
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
        print "top: ", threshold
        val_time = []
        now_score = []
        for cv,(train_val_index,test_val_index) in zip(xrange(10),skf):   

            lda = LDA()
            sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
            train_dummy_top = train_dummy_list[cv][sorted_feature]
            test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
            

            cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
            test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
            #print sorted_dic
            try: 
                train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv]) 
                train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                test_lda = lda.transform(test_dummy_top)
                test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                (train_lda, test_lda) = normalization(train_lda, test_lda)
                clf.fit(train_lda, train_label_list[cv])
                
                start = time.time()
                now_score.append(f1_score(clf.predict(test_lda), test_label_list[cv]))
                val_time.append(time.time() - start)   
            
            except LinAlgError:
                None
        exp_num = exp_num + 1
        exe_time = np.sum(val_time)
        #total_score.append(np.average(now_score))
        #total_time.append(exe_time)

        print "combine_num", combine_num
        print "score:", np.average(now_score)
        print "time:", exe_time
        result.loc[exp_num] = [combine_num, threshold, np.average(now_score), exe_time]
        result.to_csv('bot7_combine_lda_normal.csv')

combine_subset: 6
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2
combine_num 2
score: 0.360591630592
time: 0.0143284797668
top:  4
combine_num 2
score: 0.487344877345
time: 0.0137202739716
top:  6
combine_num 2
score: 0.489718614719
time: 0.0158607959747
top:  8
combine_num 2
score: 0.494163059163
time: 0.0136723518372
top:  10
combine_num 2
score: 0.51202020202
time: 0.0137157440186
top:  12
combine_num 2
score: 0.568484848485
time: 0.0131959915161
top:  14
combine_num 2
score: 0.581818181818
time: 0.0131604671478
top:  16
combine_num 2
score: 0.599393939394
time: 0.0131642818451
top:  18
combine_num 2
score: 0.599393939394
time: 0.0134482383728
top:  20
combine_num 2
score: 0.599393939394
time: 0.0125260353088
top:  22
combine_num 2
score: 0.599393939394
time: 0.011524438858
top:  24
combine_num 2
score: 0.599393939394
time: 0.0114378929138
top:  26
combine_num 2
score: 0.599393939394
time: 0.0115699768066
top:  28
combine_num 2
sco

In [33]:
test_dummy_list[0].shape[1]

61